In [48]:
import pickle
import os
import re
import numpy as np
import spacy
import tensorflow as tf
import tensorflow.keras as K
from datetime import datetime
from sklearn.preprocessing import normalize as scikit_normalize
from evaluation import plot_history
from evaluation import rmse_report
from sampling import UnderSampler
from fasttext_embedding import FastTextEmbeddingBag
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm

%matplotlib inline
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'
TB_LOG_DIR='/home/kvassay/data/z/log/E3/scalars/'
VEC_DIM=100
FASTTEXT='/home/kvassay/data/z/models/fasttext/sent_tok_lc_{}.bin'.format(VEC_DIM)
ALLOWED_SPECIAL=tuple(['?','!',':(', ':)', ':D',':-)',':-D',':\'(',':/',':-/','<3',':-P',':P'])

## Read data

In [77]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,dev,_=pickle.load(f)

CPU times: user 6.8 s, sys: 765 ms, total: 7.57 s
Wall time: 7.56 s


## Load models

In [16]:
%%time
fasttext=FastTextEmbeddingBag(FASTTEXT)

CPU times: user 9.63 s, sys: 1.93 s, total: 11.6 s
Wall time: 16.7 s


## Preprocess text + extract features
- filter out EN stop-words (and, or, ...)
- filter out non-allowed special tokens (we want to keep smileys and !,?)

In [80]:
rx_special= re.compile("|".join(re.escape(s) for s in ALLOWED_SPECIAL))

def word_filter(word):
    if word in STOP_WORDS:
        return False
    if not word.isalpha():
        if not rx_special.findall(word):
            return False
    return True

def preprocess_text(text):
    return [x.lower() for x in text if word_filter(x.lower())]

def preprocess_texts(dataset,text_keys=['summary','text']):
    for sample in tqdm(dataset):
        for key in text_keys:
            sample[key]=preprocess_text(sample[key])
    return dataset

In [81]:
%%time
train=preprocess_texts(train)
dev=preprocess_texts(dev)

100%|██████████| 8527/8527 [00:00<00:00, 24019.65it/s]

CPU times: user 20.1 s, sys: 104 ms, total: 20.2 s
Wall time: 20 s


## Extract features

In [24]:
def extract_features2(sample, fasttext):
    DEFAULT_VEC=
    vecs_summary=fasttext.forward([x.lower() for x in sample['summary']])
    vecs_text = fasttext.forward([x.lower() for x in sample['text']])
    avg_summary = np.mean(vecs_summary,axis=0)
    avg_text = np.mean(vecs_text,axis=0)
    return scikit_normalize(np.concatenate((avg_summary,avg_text), axis=0).reshape(1,-1))

SyntaxError: invalid syntax (<ipython-input-24-8e87b3ea1690>, line 2)